# Preambule

In [1]:
library(tidyverse)
library(stringr)
library(data.table)
library(stringr)
library(dplyr)
library(qs)
library(parallel)
library(clustermq)
library(ggpubr)
library(SpiecEasi)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


qs v0.23.4.



In [2]:
options(clustermq.scheduler = "sge", clustermq.template = "~/.clustermq.tmpl")

In [3]:
file.sources <- list.files('../scripts/', pattern = '*.R', full.names=TRUE, ignore.case=TRUE)
for (f in file.sources) {
    source(f)
}
rm('file.sources', 'f')

# Data

In [4]:
meta <- c('dataset','Sample','age','gender','country','BMI','westernized', 'number_reads')

In [6]:
rules <- qread('../tmp/ruleExec_full.qs')
X <- qread('../tmp/X.qs')

In [8]:
related_taxa <- qread('../tmp/related_taxa_full.qs')
is_s <- str_which(names(related_taxa), pattern = '^s_')
related_taxa[is_s] <- related_taxa[is_s] %>% lapply(function(x){str_subset(x, pattern = '^s|g')})
related_taxa$groupa <- 'groupa'
related_taxa$groupb <- 'groupb'
related_taxa$groupc <- 'groupc'
related_taxa$groupd <- 'groupd'

In [10]:
# location of the simulations
fnames <- list.files('/ebio/abt3_projects/temp_data/aruaud/MtgSimu50/p005_B10/', full.names = TRUE, pattern = 'simu')

In [11]:
length(fnames)

[1] 50

# Functions

In [13]:
getPR <- function(thr = NULL, res, related_taxa){
    
    if (is.null(thr)) {
        tmp <- res
    } else {tmp <- subset(res, val >= thr)}
    
    if (nrow(tmp) == 0){
        return(c('tp' = NA, 'fp' = NA, 'tn' = NA, 'fn' = NA) )
    }
    nodes <- unique(tmp$var)
    
    # those that should not be but are = in pred_edges but not truth
    fp <- sum(!(nodes %in% unlist(related_taxa)))
    
    # those that should be and are
    tp <- sum(sapply(related_taxa, function(x){ifelse(sum(x %in% nodes) > 0, 1, 0)}))
    
    # those that should be but are not = in truth but not in pred_edges
    fn <- length(which(!(names(related_taxa) %in% nodes)))
    
    res <- c('tp' = tp, 'fp' = fp, 'fn' = fn) 
    return(res)
    
}

In [14]:
wrapComp <- function(fname, data_ori, related_taxa, ncores = parallel::detectCores()-1, path){
    
    res <- list()
    on.exit(return(res))
    # get data 
    message('Data preparation...')
    seedOri <- as.numeric(str_extract(fname, pattern = '(?<=simu)[:digit:]+(?=\\_)'))
    res$seedOri <- seedOri
    set.seed(seedOri)
    data_ori <- data_ori[sample(1:nrow(data_ori)),]
    simu <- qread(fname)
    message(paste0("Let's go with seed ", seedOri))
    
    # ground truth
    tn <- unique( str_replace(unlist(simu$true_edges), pattern = '\\_{2}.*', replacement = '') ) 
    related_taxa <- related_taxa[tn] 
    
    te <- lapply(simu$true_edges, str_replace, pattern = '\\_{2}.*', replacement = '') 
    te <- unique(lapply(te, sort))
    expanded_edges <- list()
    for (i in 1:length(te)){
        tmp <- related_taxa[ te[[i]] ]
        tmp <- expand.grid(tmp[[1]], tmp[[2]])  
        tmp <- asplit(tmp, MARGIN=1)
        expanded_edges[[i]] <- sapply(lapply(tmp, sort), paste, collapse = ' - ')
        names(expanded_edges)[i] <- paste(sort(te[[i]]), collapse = ' - ')
    }
    res$expanded_edges <- expanded_edges
    res$tp_nodes <- length(tn)
    res$tp_edges <- length(te)
    res$n_nodes <- ncol(data_ori)
    res$n_edges <- (ncol(data_ori)^2 -ncol(data_ori))/2

    # spiec-easi
    X_counts <- select(data_ori, -c(number_reads)) 
    X_counts <- apply(X_counts, MARGIN = 2, FUN = function(x){x*data_ori$number_reads}) 
    X_p <- X_counts[which(simu$target == '1'), ]
    X_n <- X_counts[-which(simu$target == '1'), ]
    
    # small settings and gLASSO
    message('gLASSO positive...')
    glasso_p <- spiec.easi(as.matrix(X_p), method='glasso'
                      , lambda.min.ratio=1e-3, nlambda=30
                      , pulsar.params=list(rep.num=100, subsample.ratio = 0.75
                                           , seed = seedOri, ncores = parallel::detectCores()-1)
                      , verbose = TRUE)
    qsave(glasso_p, paste0(path, 'glasso_p_', seedOri, '.qs'))
    
    message('gLASSO negative')
    glasso_n <- spiec.easi(as.matrix(X_n), method='glasso'
                      , lambda.min.ratio=1e-3, nlambda=30
                      , pulsar.params=list(rep.num=100, subsample.ratio = 0.75
                                           , seed = seedOri, ncores = parallel::detectCores()-1)
                      , verbose = TRUE)
    qsave(glasso_n, paste0(path, 'glasso_n_', seedOri, '.qs'))
    
    message('Get results...')
    # get the correlation tables
    tmp <- cov2cor(getOptCov(glasso_p))
    cor_p_gl <- summary(triu(tmp*getRefit(glasso_p), k=1))
    tmp <- cov2cor(getOptCov(glasso_n))
    cor_n_gl <- summary(triu(tmp*getRefit(glasso_n), k=1))
    # replace the column numbers by their names
    cor_p_gl$i <- cor_p_gl$i %>% sapply(function(x)colnames(X_p)[x])
    cor_p_gl$j <- cor_p_gl$j %>% sapply(function(x)colnames(X_p)[x])
    cor_n_gl$i <- cor_n_gl$i %>% sapply(function(x)colnames(X_n)[x])
    cor_n_gl$j <- cor_n_gl$j %>% sapply(function(x)colnames(X_n)[x])                                    
    
    # cross 
    message('Cross...')
    in_n <- ifelse(cor_n_gl$x >= 0, '+', '-')
    in_n <- paste0(cor_n_gl$i, in_n, cor_n_gl$j)
    
    in_p <- ifelse(cor_p_gl$x >= 0, '+', '-')
    in_p <- paste0(cor_p_gl$i, in_p, cor_p_gl$j)
                                        
    cor_n_gl <- cor_n_gl[which(!(in_n %in% in_p)),]
    cor_p_gl <- cor_p_gl[which(!(in_p %in% in_n)),]
    
    res_cross <- rbind(cor_p_gl, cor_n_gl)
    res_cross$val <- res_cross$x^2
    res_cross <- summarise_all(group_by(res_cross, i,j), mean)
    res$glasso_crossed <- res_cross
    
    # PR curves
    # edges
    message('PR curves...')
    tmp <- unique(select(res_cross, c('i', 'j'))) 
    tmp <- asplit(as.matrix(tmp), MARGIN = 1)
    tmp <- sapply(tmp, function(x){paste(x, collapse = ' - ')})    
    res_cross$var <- tmp
    thr <- sort(unique(res_cross$val)) 
    
    pr_edges <- as.data.frame(t(sapply(thr, getPR, res=res_cross, related_taxa=expanded_edges)))
    pr_edges <- arrange(pr_edges, tp, fp) %>% group_by(fp) %>% summarise_all(max)
    
    res$edges <- pr_edges
    return(res)
    
}

# go

In [15]:
tmpl <- list(conda = "r-ml", cores = 20, job_time = '96:00:00', job_mem = '50G')

In [14]:
res <- Q(wrapComp
         , fail_on_error = FALSE
         , fname = fnames
         , const = list('data_ori' = X, 'related_taxa' = related_taxa
                        , 'ncores' = 80
                       , 'path' = '/ebio/abt3_projects/temp_data/aruaud/MtgSimu50/comparison_gLASSO')
         , export = c('getPR' = getPR)
         , pkgs = c('SpiecEasi', 'qs', 'stringr', 'tidyverse', 'Matrix')
         , n_jobs = 17
         , template = tmpl
         , log_worker=FALSE
        )

Warning message in (function (...) :
“Common data is 8.7 Mb. Recommended limit is  (set by clustermq.data.warning option)”
Submitting 17 worker jobs (ID: cmq8684) ...

Running 17 calculations (5 objs/8.7 Mb common; 1 calls/chunk) ...


[---------------------------------------------------]   0% (1/17 wrk) eta:  ?s

[---------------------------------------------------]   0% (2/17 wrk) eta:  ?s

[---------------------------------------------------]   0% (3/17 wrk) eta:  ?s

[---------------------------------------------------]   0% (4/17 wrk) eta:  ?s

[---------------------------------------------------]   0% (5/17 wrk) eta:  ?s

[---------------------------------------------------]   0% (6/17 wrk) eta:  ?s

[---------------------------------------------------]   0% (7/17 wrk) eta:  ?s

[==>------------------------------------------------]   6% (7/17 wrk) eta:  3d

[=====>---------------------------------------------]  12% (7/17 wrk) eta:  1d

[========>----------------------------------

In [17]:
res %>% length

[1] 50

In [18]:
qsave(res, '../tmp/comparison_glasso.qs')